# X. Model Inference

In [10]:
!pip install tensorflow
!pip install feature_engine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.4/319.4 KB 3.8 MB/s eta 0:00:00


In [18]:
import pandas as pd
import joblib
import numpy as np
from tensorflow.keras.models import load_model

## Data Loading

In [12]:
with open('full_pipeline.pkl', 'rb') as file_1:
  model_pipeline = joblib.load(file_1)

In [13]:
final_model = load_model('model_churn.h5')

In [14]:
#Membuat data baru
new_data = pd.DataFrame({
    'membership_category':['Basic Membership','Platinum Membership','Gold Membership'],
    'avg_transaction_value':[30000, 78000, 25000],
    'points_in_wallet':[200,825,500],
    'feedback': ['Too many ads', 'Poor Product Quality','No reason specified']
})
new_data

,membership_category,avg_transaction_value,points_in_wallet,feedback
0,Basic Membership,30000,200,Too many ads
1,Platinum Membership,78000,825,Poor Product Quality
2,Gold Membership,25000,500,No reason specified


### Pipeline + Model

In [15]:
#transform pipeline pada data baru
new_data_tr = model_pipeline.transform(new_data)

In [19]:
#prediksi final
y_pred = final_model.predict(new_data_tr)
y_pred = np.where(y_pred >= 0.5, 1, 0)
y_pred

1/1 [==============================] - 0s 20ms/step


array([[1],
       [0],
       [0]])

In [20]:
#hasil prediksi dijadikan dataframe
df_final = pd.DataFrame(y_pred,columns=['Churn Prediction'])
df_final

,Churn Prediction
0,1
1,0
2,0


In [21]:
#menggabungkan hasil prediksi dengan data baru
pd.concat([new_data,df_final],axis=1)

,membership_category,avg_transaction_value,points_in_wallet,feedback,Churn Prediction
0,Basic Membership,30000,200,Too many ads,1
1,Platinum Membership,78000,825,Poor Product Quality,0
2,Gold Membership,25000,500,No reason specified,0


- Pada data diatas terlihat bahwa user dengan level membership yang tergolong rendah (basic membership) ditambah lagi dengan adanya feedback negatif seperti banyaknya iklan dapat membuat user tersebut berhenti memakai produk (churn = 1)
- Pada baris kedua terlihat bahwa user tersebut memberikan feedback yang negatif (kualitas produk yang buruk) namun karena user tergolong sering menggunakan produk yang terlihat dari kategori membershipnya yaitu platinum membership maka kemungkinan user tersebut tidak akan churn (0)